In [1]:
%load_ext autoreload
%autoreload 2


In [18]:
from utils.tokenizer import load_tokenizer

tok = load_tokenizer("tokenizer")

with open("../data/small_train.txt", "r") as f:
    f.readline() # the first line is just "text"
    data = f.read()
    

first_line = data.split("\n")[0][:56]
first_tokens = tok.encode(first_line).ids

print(first_line)
print(first_tokens)
print([tok.id_to_token(i) for i in first_tokens])


"One day, a little girl named Lily found a needle in her
[5, 255, 189, 15, 68, 234, 272, 325, 185, 398, 68, 3475, 127, 153]
['"', 'One', 'day', ',', 'a', 'little', 'girl', 'named', 'Lily', 'found', 'a', 'needle', 'in', 'her']


In [69]:
with open("../data/small_train.txt", "r") as f:
    f.readline() # the first line is just "text"
    data = f.read()
    
# first 10000 words
data = " ".join(data.split()[:10000])
    
data = data.lower()
data = data.replace(".", "")
data = data.replace(",", "")
data = data.replace("!", "")
data = data.replace("?", "")
data = data.replace("'", "")
data = data.replace("\"", "")

word_to_ids = {word: i for i, word in enumerate(set(data.split()))}
ids_to_word = {i: word for word, i in word_to_ids.items()}

In [70]:
tokenized_data = [word_to_ids[word] for word in data.split()]

In [72]:
from utils import NGramTrie

trie = NGramTrie.fit(tokenized_data, ngram_max_length=3)

In [96]:
trie.search([word_to_ids[word] for word in "one day".split()]), data.count("one day")

(56, 56)

In [97]:
trie.search([word_to_ids[word] for word in "one day a".split()]), data.count("one day a")

(11, 11)

In [98]:
trie.search([word_to_ids[word] for word in "one day a".split()], "++*"), data.count("one day a")

(56, 11)

In [101]:
node = trie.find_all_nodes([word_to_ids[word] for word in "one day".split()])
[(ids_to_word[token], node.count) for token, node in node[0].children.items()]

[('a', 11),
 ('beep', 1),
 ('the', 7),
 ('while', 1),
 ('tim', 6),
 ('max', 1),
 ('when', 2),
 ('timmy', 1),
 ('his', 2),
 ('sue', 1),
 ('she', 8),
 ('he', 4),
 ('it', 1),
 ('zigzag', 1),
 ('they', 1),
 ('her', 1),
 ('tims', 1),
 ('joe', 1),
 ('mia', 1),
 ('lucy', 1),
 ('charlies', 1),
 ('sam', 1),
 ('bill', 1)]

In [106]:
data.count("one day while"), data.count("one day tim"), data.count("one day she")  # why is tim wrong?

(1, 8, 8)